# 4.2.1: 2乗和誤差(SSE:Sum of Square Error)

https://docs.google.com/presentation/d/14gYge5fHH5wI9b3uyyJY1jM-5I9E1jGMVKQecCEUD9Q/edit#slide=id.g10e449d0a8e_0_37

In [1]:
import numpy as np

In [2]:
# sum of squared error
def sum_squared_error(y,t):
    return 0.5 * np.sum((y-t)**2)

In [3]:
y = [0.1,0.05,0.6,0.0,0.05,0.1,0.0,0.1,0.0,0.0]
t = [0,0,1,0,0,0,0,0,0,0] # 正解ラベルを1としてそれ以外は0で表す "one-hot表現"

In [4]:
sum_squared_error(np.array(y),np.array(t))

0.09750000000000003

In [5]:
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]

In [6]:
print('y=',y, '\n'
      't=',t)

y= [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0] 
t= [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]


In [7]:
sum_squared_error(np.array(y),np.array(t))

0.5975

# 4.2.2 交差エントロピー誤差 (Cross Entropy Error)

https://docs.google.com/presentation/d/14gYge5fHH5wI9b3uyyJY1jM-5I9E1jGMVKQecCEUD9Q/edit#slide=id.g10e449d0a8e_0_46

In [8]:
def cross_entropy_error(y,t):
    delta = 1e-7 # np.log(0)でのエラーの発生(-inf:マイナス無限大)を防ぐ
    return -np.sum(t*np.log(y + delta))

In [9]:
np.log(0)

/var/folders/40/5ckj9ggn49q3nfcs0qc0cln00000gn/T/ipykernel_35826/2933082444.py:1: RuntimeWarning: divide by zero encountered in log
  np.log(0)


-inf

In [10]:
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]

In [11]:
cross_entropy_error(np.array(y),np.array(t))

0.510825457099338

In [12]:
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]

In [13]:
cross_entropy_error(np.array(y),np.array(t))

2.302584092994546

# 4.2.3 ミニバッチ学習

In [14]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize=True, one_hot_label=True) 

print(x_train.shape) # (60000, 784), 28x28の画像データ
print(t_train.shape) # (60000, 10)

(60000, 784)
(60000, 10)


In [15]:
# ランダムに10枚抜き出す
train_size = x_train.shape[0] # 60000
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

In [16]:
x_batch

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [17]:
t_batch

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [18]:
print(x_batch.shape,'\n', t_batch.shape)

(10, 784) 
 (10, 10)


# 4.2.4 バッチ対応版　交差エントロピー誤差

In [19]:
#訓練データがone-hotの場合
def _cross_entropy_error(y, t):
    if y.ndim == 1: # Yの次元数が1の場合
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size) 
    batch_size = y.shape[0] 
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

In [20]:
#訓練データがラベルとして与えられた場合
def cross_entropy_error(y, t):
    if y.ndim == 1: # Yの次元数が1
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

np.arange(batch_size) は 、0 か ら batch_size-1 ま で の 配 列 を 生 成 します。たとえば、batch_size が 5 だとしたら、np.arange(batch_size) は [0, 1, 2, 3, 4] の NumPy 配 列 を 生 成 し ま す 。t に は ラ ベ ル が [2, 7, 0, 9, 4] のように格納されているので、y[np.arange(batch_size), t] は 、各 デ ー タ の 正 解 ラ ベ ル に 対 応 す る ニ ュ ー ラ ル ネ ッ ト ワ ー ク の 出 力 を 抽 出 し ま す（ こ の 例 で は 、y[np.arange(batch_size), t] は 、 [y[0,2], y[1,7], y[2,0], y[3,9], y[4,4]] の NumPy 配列を生成します）。
(Page 95). 

## 教師データが one-hot の場合 (batch_size = 3)

In [21]:
# (仮の)出力yを作成
y = np.array([
    [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0], 
    [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1], 
    [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
])
print(y)
print(y.shape)

[[0.1  0.05 0.6  0.   0.05 0.1  0.   0.1  0.   0.  ]
 [0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1 ]
 [0.1  0.05 0.1  0.   0.05 0.1  0.   0.6  0.   0.  ]]
(3, 10)


In [22]:
# (仮の)教師データtを作成 (one-hot)
t = np.array([
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], 
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], 
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
])
print(t)
print(t.shape)

[[0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 1 0 0 0 0 0 0 0]]
(3, 10)


In [23]:
# 微小な値を設定
delta = 1e-7

# 出力の対数をとる
log_y = np.log(y + delta)
print(np.round(log_y, 2))
# log y(n,k) の計算を行うことで、全ての要素が0以下の値になりました。

[[ -2.3   -3.    -0.51 -16.12  -3.    -2.3  -16.12  -2.3  -16.12 -16.12]
 [ -2.3   -2.3   -2.3   -2.3   -2.3   -2.3   -2.3   -2.3   -2.3   -2.3 ]
 [ -2.3   -3.    -2.3  -16.12  -3.    -2.3  -16.12  -0.51 -16.12 -16.12]]


In [24]:
# 教師データと出力の対数の積を計算
tmp = t * log_y
print(np.round(tmp, 2))
# t(n,k) * log y(n,k) の計算を行うことで正解ラベルの値だけ残る

[[-0.   -0.   -0.51 -0.   -0.   -0.   -0.   -0.   -0.   -0.  ]
 [-0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -2.3  -0.  ]
 [-0.   -0.   -2.3  -0.   -0.   -0.   -0.   -0.   -0.   -0.  ]]


In [25]:
# 教師データと出力の対数の積の和を計算
tmp = np.sum(t * np.log(y + delta))
print(tmp)

-5.11599364308843


In [26]:
# バッチサイズを取得
batch_size = y.shape[0]
print('batch_size=', batch_size)

# 交差エントロピー誤差を計算:式(4.3)
E = - np.sum(t * np.log(y + delta)) / batch_size
print('Cross Entropy Error=', E)

batch_size= 3
Cross Entropy Error= 1.70533121436281


### 教師データがスカラーの場合

In [27]:
A = np.arange(50).reshape(5, 10)  # np.arange()は指定個数の配列を作成
print(A)
print(A.shape)

[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [20 21 22 23 24 25 26 27 28 29]
 [30 31 32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47 48 49]]
(5, 10)


In [28]:
# 行を抽出
print(A[2, :])

# 列を抽出
print(A[:, 3])

# 要素を抽出
print(A[2, 3])

[20 21 22 23 24 25 26 27 28 29]
[ 3 13 23 33 43]
23


In [29]:
# 抽出する行番号を指定
row_idx = np.array([2, 1, 4, 3, 2])

# 抽出する列番号を指定
col_idx = np.array([4, 8, 2, 5, 0])

# 要素を抽出
print(A[row_idx, col_idx])

[24 18 42 35 20]


In [30]:
y

array([[0.1 , 0.05, 0.6 , 0.  , 0.05, 0.1 , 0.  , 0.1 , 0.  , 0.  ],
       [0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 ],
       [0.1 , 0.05, 0.1 , 0.  , 0.05, 0.1 , 0.  , 0.6 , 0.  , 0.  ]])

In [31]:
t

array([[0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]])

In [32]:
# 各データの正解ラベルを抽出
t = np.argmax(t, axis=1)
print(t)
#t = np.array([
#    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], 
#    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], 
#    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
#])

[2 8 2]


In [33]:
# バッチサイズを抽出
batch_size = y.shape[0]

# 出力から各データの正解ラベルに関する要素を抽出
print(y[np.arange(batch_size), t])

[0.6 0.1 0.1]


In [34]:
# 各データの交差エントロピー誤差を計算
E_n = - np.log(y[np.arange(batch_size), t] + delta)
print(E_n)

[0.51082546 2.30258409 2.30258409]


In [35]:
# 交差エントロピー誤差を計算:式(4.2)
E = - np.sum(np.log(y[np.arange(batch_size), t] + delta)) / batch_size
print(E)

1.70533121436281
